# Action recognition using TAO ActionRecognitionNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train 3D RGB only for action recognition on the merl-shopping (https://www.merl.com/demos/merl-shopping-dataset) dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .etlt file for deployment to DeepStream.

## Table of Contents

This notebook shows an example usecase of ActionRecognitionNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
%env HOST_DATA_DIR=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/merl_data_notebook
# note: You could set the HOST_SPECS_DIR to folder of the experiments specs downloaded with the notebook
%env HOST_SPECS_DIR=/home/mjhuria/tao_experiments/ar_merl/specs
%env HOST_RESULTS_DIR=/home/mjhuria/tao_experiments/ar_merl/ar_merl_results

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
       {
           "source": os.path.expanduser("~/.cache"),
           "destination": "/root/.cache"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

We will be using the merl retail dataset. https://www.merl.com/demos/merl-shopping-dataset. Assuming the data has label folder which contains .mat files and video folder which has .mp4 files. User can split the data for training and validation. 

In [ ]:
## The Merl dataset is for action detection. To make it work with action recognition we need to do some preprocessing. Please run the
#the code below. Eventually you will get three separate folders for test,train and val. The dir structure should be:
#<data_split>/<name_class>/<clips_i>/rgb:
! python convert.py --data_split val --labels_path=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/labels_val --video_path=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/videos_val --start=1 --end=10
! python convert.py --data_split train --labels_path=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/labels_train --video_path=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/videos_train --start=1 --end=65
! python convert.py --data_split test --labels_path=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/labels_test --video_path=/home/mjhuria/tao/tao_usecases/ar_custom/merl-shopping/videos_test --start=1 --end=31

In [ ]:
! ls train/Hand_in_Shelf
! ls train/Reach_to_Shelf
! ls train/Retract_from_Shelf

### 2.2 Download pretrained model 

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [ ]:
# Pull pretrained model 
!wget https://github.com/piergiaj/pytorch-i3d/raw/master/models/rgb_imagenet.pt
!mv rgb_imagenet.pt $HOST_RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained

## 3. Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training parameters including:

* model_config: configure the model setting
    * model_type: type of model, rgb/of/joint
    * backbone: resnet18/34/50/101/152 
    * rgb_seq_length: length of RGB input sequence
    * input_type: 2d/3d
    * sample_strategy: consecutive
    * dropout_ratio: probability to drop the hidden units
* train_config: configure the training hyperparameters
    * optim_config
    * epochs
    * checkpoint_interval
* dataset_config: configure the dataset and augmentation methods
    * train_dataset_dir
    * val_dataset_dir
    * label_map: map the class label to id
    * output_shape
    * batch_size
    * workers: number of workers to do data loading
    * clips_per_video: number of clips to be sampled from single video
    * augmentation_config

Please refer to the TAO documentation about ActionRecognitionNet to get all the parameters that are configurable.

In [ ]:
!cat $HOST_SPECS_DIR/train_rgb_i3d.yaml

## 4. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

### 4.1 Train 3D model:

With the pretrained model, we can even get better accuracy with less epochs.

`KNOWN ISSUE`: 
- 1) The training log will be corrupted by pytorch warning in the notebook:

     `[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)` 
     
     To see the full log in std out, please run the command in terminal. 
- 2) "=" in the checkpoint file name should removed before using the checkpoint in command.

In [ ]:
print("Train RGB only model with PTM")
!tao action_recognition train \
                  -e $SPECS_DIR/train_rgb_i3d.yaml \
                  -r $RESULTS_DIR/rgb_3d \
                  -k $KEY \
                  model_config.rgb_pretrained_model_path=$RESULTS_DIR/pretrained/rgb_imagenet.pt # \
                  model_config.rgb_pretrained_num_classes=5

In [ ]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm

In [ ]:
print('Rename a model: Note that the training is not deterministic, so you may change the model name accordingly.')
print('---------------------')
# NOTE: The following command may require `sudo`. You can run the command outside the notebook.
!mv $HOST_RESULTS_DIR/rgb_3d/ar_model_epoch=64-val_loss=0.50.tlt $HOST_RESULTS_DIR/rgb_3d/rgb_only_model.tlt 
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d/rgb_only_model.tlt

## 5. Evaluate trained models <a class="anchor" id="head-4"></a>

We provide two different sample strategy to evaluate the pretrained model on video clips.

* `center` mode: pick up the middle frames of a sequence to do inference. For example, if the model requires 32 frames as input and a video clip has 128 frames, then we will choose the frames from index 48 to index 79 to do the inference. 
* `conv` mode: convolutionly sample 10 sequences out of a single video and do inference. The final results are averaged.

Evaluate RGB model trained with PTM

In [ ]:
!tao action_recognition evaluate \
                    -e $SPECS_DIR/evaluate_rgb_i3d.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d/rgb_only_model.tlt  \
                    batch_size=1 \
                    test_dataset_dir=$DATA_DIR/test \
                    video_eval_mode=conv

## 6. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the action recognition inference tool to generate inferences with the trained RGB models and print the results. 

There are also two modes for inference just like evaluation: `center` mode and `conv` mode. And the final output will show each input sequence label in the videos like:
`[video_sample_path] [labels list for sequences in the video sample]`

In [ ]:
!tao action_recognition inference \
                    -e $SPECS_DIR/infer_rgb_i3d.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d/rgb_only_model.tlt \
                    inference_dataset_dir=$DATA_DIR/test/Hand_in_Shelf \
                    video_inf_mode=conv

## 7. Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/export

In [ ]:
# Export the RGB model to encrypted ONNX model
!tao action_recognition export \
                   -e $SPECS_DIR/export_rgb_i3d.yaml \
                   -k $KEY \
                   model=$RESULTS_DIR/rgb_3d/rgb_only_model.tlt\
                   output_file=$RESULTS_DIR/export/rgb_i3d_100epoc.etlt

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

This notebook has come to an end. You may continue by deploying this model to [DeepStream](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_3D_Action.html)